In [1]:
import pickle

drv = None
with open("drv.pkl", "wb") as f:
    pickle.dump(drv, f)

In [1]:
%reload_ext autoreload
%autoreload 2

import h5py
N_Cs = 9

with h5py.File('../dxtb/dxtb-gpu/gpu-cpu_analysis/rdkit/alkanes_data_500.hdf5', 'r') as f:
    for mol_name, data in f.items():
        if mol_name == f"alkane_{N_Cs}_carbons":
            atomic_numbers = data['atomic_numbers'][:]
            coordinates = data['coordinates'][:]

print(f"Number of carbon atoms in {mol_name}: {N_Cs}")
print(f"Nb of atoms: {len(atomic_numbers)}")

Number of carbon atoms in alkane_9_carbons: 9
Nb of atoms: 29


In [2]:
import dxtb
from dxtb._src.typing import DD
import torch
from ase.build import molecule
from dxtb.config import ConfigCache

opts = {"scf_mode": "implicit", "batch_mode": 1, "int_driver": "libcint", "maxiter": 1}
batch_size = 64
results = {}

print(f"Number of carbon atoms in {mol_name}: {N_Cs}")
print(f"Nb of atoms: {len(atomic_numbers)}")
print(f"batch_size: {batch_size}")
print(f"opts: {opts}")

for device in ["cuda:0", "cpu"]:
    print(f"\nDevice: {device}")
    dd = {"dtype": torch.float32, "device": torch.device(device)}
    numbers = torch.tensor(atomic_numbers, device=dd["device"], dtype=torch.int32)
    positions = torch.tensor(coordinates, device=dd["device"], dtype=dd["dtype"])
    numbers = torch.stack([numbers] * batch_size)
    positions = torch.stack([positions] * batch_size).requires_grad_()
    charges = torch.zeros((batch_size,), device=dd["device"], dtype=dd["dtype"])

    calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, **dd, opts=opts, timer=True)
    calc.opts.cache = ConfigCache(enabled=False, density=True, fock=True, overlap=False)

    
    dxtb.timer.reset()
    e = calc.get_energy(positions, chrg=charges)
    dxtb.timer.start("Forces autograd")
    forces = torch.autograd.grad(sum(e), positions, retain_graph=True)[0]
    dxtb.timer.stop("Forces autograd")
    dxtb.timer.print(v=0)

    results[device] = {
        "energy": e.detach().cpu(),
        "forces": forces.detach().cpu()
    }

# Compare results
energy_diff = (results["cuda:0"]["energy"] - results["cpu"]["energy"]).abs().max()
forces_diff = (results["cuda:0"]["forces"
] - results["cpu"]["forces"]).abs().max()

print(f"\n[Comparison]")
print(f"GPU energy: {results['cuda:0']['energy'].mean().item():.6e}")
print(f"CPU energy: {results['cpu']['energy'].mean().item():.6e}")
print(f"Max energy diff: {energy_diff.item():.6e}")
print(f"Max forces diff: {forces_diff.item():.6e}")

Number of carbon atoms in alkane_9_carbons: 9
Nb of atoms: 29
batch_size: 64
opts: {'scf_mode': 'implicit', 'batch_mode': 1, 'int_driver': 'libcint', 'maxiter': 1}

Device: cuda:0


Timings
-------

Objective                Time (s)        % Total
------------------------------------------------
Classicals                  0.045           6.93
 - DispersionD3             0.033          74.06
 - Halogen                  0.005          11.58
 - Repulsion                0.006          14.07
Integrals                   0.239          36.87
 - Overlap                  0.236          98.60
 - Core Hamiltonian         0.003           1.39
SCF                         0.104          15.99
 - Interaction Cache        0.001           0.93
 - Potential                0.050          47.97
 - Fock build               0.000           0.12
 - Diagonalize              0.038          36.65
 - Density                  0.001           0.53
 - Charges                  0.000           0.46
Forces autograd  